## GeoCat experimental

In [38]:
import importlib

import numpy as np
import pandas as pd
import recommenders as rec
import geocat
import networkx as nx
import matplotlib.pyplot as plt
import areamanager
import math
import timeit
from IPython.core.debugger import set_trace
%matplotlib inline
def string_to_array(string):
    if string == '':
        return list()
    return eval(string)
importlib.reload(geocat)
CITY="madison"



In [2]:
df_checkin=pd.read_csv("../data/checkin/cities/"+CITY+".csv",converters={'categories':string_to_array})
df_checkin['date']=pd.to_datetime(df_checkin['date'])
# users=df_checkin['user_id'].drop_duplicates().reset_index(drop=True)
# num_users=len(users)
df_checkin.set_index("user_id",inplace=True)
df_checkin.head()

,business_id,date,latitude,longitude,categories
user_id,,,,,
MB7Z80PJxNyQM7keDpR2XQ,br1uQbc3YU5jCpYi46nIgQ,2018-04-04 00:09:03,43.069425,-89.389567,"[Coffee & Tea, Cafes]"
4HwS2oCVQ1-f_kzHCv5X3A,br1uQbc3YU5jCpYi46nIgQ,2018-02-16 20:40:41,43.069425,-89.389567,"[Coffee & Tea, Cafes]"
OLBH2Ds1wllrzpHlyojzJQ,br1uQbc3YU5jCpYi46nIgQ,2018-04-12 11:31:03,43.069425,-89.389567,"[Coffee & Tea, Cafes]"
wd3xoNaDLib8dhQ7BxUl6g,br1uQbc3YU5jCpYi46nIgQ,2018-03-07 14:08:41,43.069425,-89.389567,"[Coffee & Tea, Cafes]"
4HwS2oCVQ1-f_kzHCv5X3A,br1uQbc3YU5jCpYi46nIgQ,2018-02-16 20:48:49,43.069425,-89.389567,"[Coffee & Tea, Cafes]"


In [3]:
df_poi=pd.read_csv("../data/poi/"+CITY+".csv",converters={'categories':string_to_array})
df_poi=df_poi.set_index("business_id")

In [4]:
df_poi.head(1)

,latitude,longitude,categories,subarea_id
business_id,,,,
nkoie_y3fbaQGjZqFGqsrg,42.98642,-89.547928,"[Auto Glass Services, Windows Installation, Wi...",829.0


# Test and training set

In [5]:
df_checkin=df_checkin.sort_values(['user_id','date'])

ranks=df_checkin.groupby('user_id')['date'].rank(method='first').reset_index(drop=True)
ranks.head()

#ranks=df_review.groupby('user_id')['date'].rank(method='first')
count=df_checkin.reset_index()['user_id'].map(df_checkin.groupby('user_id')['date'].apply(len))
count.head()

training_set_delimiter=(ranks/count)<0.72
test_set_delimiter=np.logical_not(training_set_delimiter)
training_set_delimiter

df_checkin_train=df_checkin.reset_index()[training_set_delimiter]
df_checkin_train.count()

df_checkin_test=df_checkin.reset_index()[test_set_delimiter]
df_checkin_test.count()

users=df_checkin_train['user_id'].drop_duplicates().reset_index(drop=True)



In [6]:
dict_alias_title,category_tree,dict_alias_depth=geocat.category_manipulation_utils()
undirected_category_tree=category_tree.to_undirected()

In [7]:
num_users=len(users)
users.count()

1022

In [8]:
#df_checkin_train[['user_id','business_id','date']].groupby(['user_id','business_id']).count().reset_index().to_csv("../data/checkin/train/"+CITY+".csv",index=False,header=False)

#df_checkin_test[['user_id','business_id','date']].groupby(['user_id','business_id']).count().reset_index().to_csv("../data/checkin/test/"+CITY+".csv",index=False,header=False)

In [9]:
df_checkin_train=df_checkin_train.set_index('user_id')
df_checkin_test=df_checkin_test.set_index('user_id')

In [10]:
series_poi_checkin=df_checkin_train.reset_index().groupby('business_id').count()['user_id']
series_poi_checkin.name="checkin"
series_poi_checkin.head(2)

business_id
-0Hj1hb_XW6ybWq2M7QhGA    4
-0b86isaXMY0v4g-V8GZ9Q    3
Name: checkin, dtype: int64

In [11]:
series_poi_checkin.loc["FEH0i7SnTH81k451ZadKIQ"]

99

### load poi neighbor

In [12]:
df_poi_neighbor=pd.read_csv("../data/neighbor/"+CITY+".csv",converters={'neighbors':string_to_array})
df_poi_neighbor=df_poi_neighbor.set_index("business_id")

In [13]:
df_checkin.loc["scaxsfOynKUZEijn_UCTkg"].count()


business_id    46
date           46
latitude       46
longitude      46
categories     46
dtype: int64

In [14]:
user_id="scaxsfOynKUZEijn_UCTkg"
df_checkin.loc[[user_id]].head(2)


,business_id,date,latitude,longitude,categories
user_id,,,,,
scaxsfOynKUZEijn_UCTkg,OSXb-DOITpDR7nm8wjrZ2w,2017-02-28 00:05:42,43.071577,-89.524585,[Nail Salons]
scaxsfOynKUZEijn_UCTkg,U7G4VBrra439LgGBvV7dcg,2017-02-28 16:23:37,43.123634,-89.350392,[Eyelash Service]


In [ ]:
N=80
K=20
VERY_SMALL_VALUE=-100
i=0

# tmp_rec_list=rec.mostpopular(df_checkin,K)
# tmp_rec_list['score']=tmp_rec_list['score']/tmp_rec_list['score'].max()
# print(tmp_rec_list)
dict_rec_list=dict()
for user_id in users:
    user_id="scaxsfOynKUZEijn_UCTkg"
    print(user_id)
    # User visits
    df_user_checkin=df_checkin_train.loc[[user_id]].reset_index(drop=True)
    
    ####print(df_user_checkin['business_id'])
    
    list_user_pois=set(df_user_checkin['business_id'].drop_duplicates().values)
    
    
    tmp_rec_list=rec.mostpopular(df_checkin_train.reset_index(),N,num_users,list_user_pois)
    
    # add other columns
    tmp_rec_list=pd.merge(tmp_rec_list,df_poi.reset_index(),on='business_id').drop_duplicates(subset=df_poi.reset_index().columns.difference(['categories'])).reset_index(drop=True)

    business_cover=dict()
    for business_id in df_user_checkin['business_id'].drop_duplicates().reset_index(drop=True):
        business_cover[business_id]=0
    # Constants
    #COVER_OF_POI=len(df_user_review)/len(tmp_rec_list)
    div_geo_cat_weight = 0.5 # beta,this is here because of the work to be done on parameter customization for each user
    div_weight = 0.75 # lambda, geo vs cat
    current_proportionality=0
    
    rec_list=pd.DataFrame(columns=['business_id','score','latitude','longitude','categories'])
#     relevant_categories_user=geocat.relevant_categories_to_the_user(df_user_checkin)

    # REPLICATE WRONG CODE
    series_user_checkin=df_user_checkin.groupby('business_id').count()['date']
    series_user_checkin.name="checkin"
    ####print(series_poi_checkin[business_cover.keys()])
    # This code is to replicate the error in original geocat
    ####mean_poi_visits=(series_poi_checkin[business_cover.keys()]*series_user_checkin).sum()/series_user_checkin.sum()
    # using entry from java
    mean_poi_visits=pd.read_csv("~/checkinjava.csv").set_index("business_id")['checkin'].mean()
    # Relevant categories

    relevant_categories = set()
    print(mean_poi_visits)
    #pd.read_csv("~/checkinjava.csv").set_index("business_id")['checkin']
    ####for poi_id,checkins in series_poi_checkin[business_cover.keys()].iteritems():
    
    for poi_id,checkins in pd.read_csv("~/checkinjava.csv").set_index("business_id")['checkin'].iteritems():
        # Check if poi is relevant
        if checkins >= mean_poi_visits:
            # add relevant categories
            print(poi_id)
            print(df_poi.loc[poi_id]['categories'])
            for category in df_poi.loc[poi_id]['categories']:
                relevant_categories.add(category)
        
        

    print("Number of relevant categories:",len(relevant_categories))#,"Number of relevant cat dupl:",cc)
    # REPLICATE WRONG CODE
    #for i in relevant_categories:
   #     print(dict_alias_title[i])
    #print(relevant_categories)
    
    
    
    
    # post-processing for diversification
    for i in range(K):
#         start = timeit.default_timer()
        #num_current_candidates=len(tmp_rec_list)
       # set_trace()
        poi_to_insert=None
        max_objective_value=VERY_SMALL_VALUE
        rec_list_categories=geocat.category_in_rec_list(tmp_rec_list)

        for index,poi_candidate in tmp_rec_list.iterrows():
            candidate_neighbors=df_poi_neighbor.loc[poi_candidate.business_id].neighbors
            poi_neighbors=df_poi.loc[set(candidate_neighbors) & list_user_pois]
            
            objective_value=geocat.objective_ILD_GC_PR(poi_candidate,df_user_checkin,rec_list,K,\
                                                business_cover,current_proportionality,\
                                               div_geo_cat_weight,div_weight,dict_alias_title,undirected_category_tree,\
                                                      relevant_categories,rec_list_categories,poi_neighbors)
            break
            if objective_value > max_objective_value:
                max_objective_value=objective_value
                poi_to_insert=poi_candidate 
        break
#         stop = timeit.default_timer()
#         print('Time:', stop - start)
        if poi_to_insert is not None:
            poi_neighbors=df_poi_neighbor.loc[poi_to_insert.business_id].neighbors
            poi_to_insert['score']=max_objective_value
            rec_list=rec_list.append(poi_to_insert)
            # remove from tmp_rec_list
            tmp_rec_list=tmp_rec_list[tmp_rec_list['business_id']!=poi_to_insert['business_id']].reset_index(drop=True)
            
            current_proportionality=geocat.update_geo_cov(poi_to_insert,df_user_checkin,K,business_cover,poi_neighbors)
    
    dict_rec_list[user_id]=rec_list[['business_id','score']]
    print(rec_list[['business_id','score']])
    break
    i+=1
    if i % num_users==0:
        print(i)

In [ ]:
pd.read_csv("~/teste.csv",header=None)

In [18]:
dict_alias_title['']

'gourmet'

In [ ]:
pd.merge(pd.read_csv("~/a.txt",header=None),
pd.read_csv("~/b.txt",header=None),how='inner')